In [1]:
import optuna
import yaml
import time
import numpy as np
import pandas as pd
import torch
import multiprocessing
from multiprocessing import Pool

from ili.dataloaders import StaticNumpyLoader
from ili.inference import InferenceRunner
from ili.validation import ValidationRunner
import ili

from CASBI.utils.create_dataframe import rescale

device='cuda'

N_subhalos = 2
data = pd.read_parquet('../../../../data/dataframe/dataframe.parquet')
data = rescale(data, mean_and_std_path='../../../../data/preprocess/mean_and_std.parquet', scale_observations=True, scale_parameter=True, inverse=True) 
data =  data.drop(['gas_log10mass', 'a','redshift', 'mean_metallicity', 'std_metallicity','mean_FeMassFrac', 'std_FeMassFrac', 'mean_OMassFrac', 'std_OMassFrac'], axis=1)
min_feh, max_feh = min(data['feh']), max(data['feh'])
min_ofe, max_ofe = min(data['ofe']), max(data['ofe'])
conditions = data[data.columns.difference(['feh', 'ofe', 'Galaxy_name'], sort=False)].drop_duplicates()

minimum_theta = [conditions[col].values.min() for col in conditions.columns]   
maximum_theta = [conditions[col].values.max() for col in conditions.columns]       
minimum_theta = np.array(minimum_theta)
maximum_theta = np.array(maximum_theta)
def repeat_array(arr, repetitions):
    return np.repeat(arr, repetitions)
repeat_minimum_theta = repeat_array(minimum_theta, N_subhalos)
repeat_maximum_theta = repeat_array(maximum_theta, N_subhalos) 

prior = ili.utils.Uniform(low=repeat_minimum_theta, high=repeat_maximum_theta, device=device)

N_test = 1_000
def preprocess_testset(i):
    galaxies = set(data['Galaxy_name'].drop_duplicates().sample(N_subhalos, random_state=i))
    parameters =  data[data['Galaxy_name'].isin(galaxies)].drop(['feh', 'ofe', 'Galaxy_name'], axis=1).drop_duplicates().values.T.reshape(-1)
    galaxy_data = data[data['Galaxy_name'].isin(galaxies)].values
    histogram_galaxy, _, _ = np.histogram2d(galaxy_data[:, 0], galaxy_data[:, 1], bins=64, range=[[min_feh, max_feh], [min_ofe, max_ofe]])
    sim_data =  np.expand_dims(np.log10(histogram_galaxy + 1e-6 +1), axis=0)
    return parameters, sim_data, galaxies

# Create a pool of workers
with Pool() as pool:
    # Map the function to the data
    results = pool.map(preprocess_testset, range(N_test))
    
# Unpack the results
theta_test, x_test, galaxies_test = zip(*results)

#take the first test set element as x_0 and theta_0    
galaxies_0 = galaxies_test[0]
data_to_plot_halos = data[data['Galaxy_name'].isin(galaxies_0)].to_parquet('./halos_0.parquet')
theta_0 =  theta_test[0]
x_0 =  x_test[0]

N = 10_000
def process_sample(i):
    galaxies = data['Galaxy_name'].drop_duplicates().sample(N_subhalos, random_state=i+int(time.time()))
    while (any(set(galaxies) == galaxy_in_testset for galaxy_in_testset in galaxies_test)):
        print('matched galaxies, try again')
        print('galaxies', set(galaxies))
        print('test galaxies', galaxies_test)
        galaxies = data['Galaxy_name'].drop_duplicates().sample(N_subhalos, random_state=i)
    parameters =  data[data['Galaxy_name'].isin(galaxies)].drop(['feh', 'ofe', 'Galaxy_name'], axis=1).drop_duplicates().values.T.reshape(-1)
    galaxy_data = data[data['Galaxy_name'].isin(galaxies)].values
    histogram_galaxy, _, _ = np.histogram2d(galaxy_data[:, 0], galaxy_data[:, 1], bins=64, range=[[min_feh, max_feh], [min_ofe, max_ofe]])
    sim_data =  np.expand_dims(np.log10(histogram_galaxy + 1e-6 +1), axis=0)
    return parameters, sim_data

# Create a pool of workers
with Pool() as pool:
    # Map the function to the data
    results = pool.map(process_sample, range(N))

# Unpack the results
theta, x = zip(*results)

#save in .npy files, we remove the first element of the test set since it will be stored as x_0 and theta_0
np.save('./x_test.npy', x_test[1:])
np.save('./theta_test.npy', theta_test[1:])
np.save('./x_0.npy', x_0)
np.save('./theta_0.npy', theta_0)
np.save('./x.npy', x)
np.save('./theta.npy', theta)
print('finish prepare the data')


finish prepare the data


In [2]:
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, input_channel, output_dim):
        super(ConvNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_channel, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        out = self.conv_layers(x)
        out = out.view(out.size(0), -1)
        out = self.fc_layers(out)
        return out

In [3]:
def objective(trial, device):
    # Suggest values for the hyperparameters
    model = trial.suggest_categorical('model', ['maf', 'nsf'])
    hidden_features = trial.suggest_categorical('hidden_features', [10, 50, 70, 100])
    num_transforms = trial.suggest_categorical('num_transforms', [5, 10, 15, 20])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 5e-5, 1e-4]) #suggest_loguniform('learning_rate', 1e-5, 1e-4)
    output_dim = trial.suggest_categorical('output_dim', [5, 10, 32, 64])
        
    # reload all simulator examples as a dataloader
    all_loader = StaticNumpyLoader.from_config("./data.yaml")
    
    embedding_net = ConvNet(input_channel=1, output_dim=output_dim)
    
    nets = [ ili.utils.load_nde_sbi(engine='NPE', model=model, hidden_features=hidden_features, num_transforms=num_transforms, embedding_net=embedding_net)]

    train_args = {
    'training_batch_size': 1024,
    'learning_rate': learning_rate,
    'stop_after_epochs': 20}
    
    runner = InferenceRunner.load(
    backend='sbi',
    engine='NPE',
    prior=prior,
    nets=nets,
    device=device,
    embedding_net=embedding_net,
    train_args=train_args,
    proposal=None,
    out_dir=None,)

    # train a model to infer x -> theta. save it as toy/posterior.pkl
    # runner = InferenceRunner.from_config(f"./training.yaml")
    _, summaries = runner(loader=all_loader)
    
    return summaries[0]['validation_log_probs'][-1]

In [4]:
study_name = 'example_study'  # Unique identifier of the study.
storage_name = 'sqlite:///example.db'
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)

[I 2024-05-29 16:10:05,049] Using an existing study with name 'example_study' instead of creating a new one.


In [9]:
import torch
import multiprocessing

def optimize_on_device(device_id):
    device = f'cuda:{device_id}'
    study.optimize(lambda trial: objective(trial, device), n_trials=100)

if __name__ == '__main__':
    current_start_method = multiprocessing.get_start_method()
    if current_start_method is None or current_start_method.lower() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    num_gpus = torch.cuda.device_count()
    with multiprocessing.Pool(num_gpus) as p:
        p.map(optimize_on_device, range(num_gpus))

Process SpawnPoolWorker-134:
Process SpawnPoolWorker-130:
Process SpawnPoolWorker-129:
Process SpawnPoolWorker-131:
Process SpawnPoolWorker-132:
Process SpawnPoolWorker-135:
Process SpawnPoolWorker-133:
Traceback (most recent call last):
  File "/export/home/vgiusepp/miniconda3/envs/fff/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/export/home/vgiusepp/miniconda3/envs/fff/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/export/home/vgiusepp/miniconda3/envs/fff/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/export/home/vgiusepp/miniconda3/envs/fff/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'optimize_on_device' on <module '__main__' (built-in)>
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 